In [1]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import numpy as np

In [2]:
redmapper_catalog = Table.read("/home/kapnadak/bliss/case_studies/galaxy_clustering/notebooks/redmapper_sva1-expanded_public_v6.3_members.fits").to_pandas()
redmapper_catalog

,ID,RA,DEC,R,P,P_FREE,THETA_I,THETA_R,MAG_AUTO_G,MAGERR_AUTO_G,MAG_AUTO_R,MAGERR_AUTO_R,MAG_AUTO_I,MAGERR_AUTO_I,MAG_AUTO_Z,MAGERR_AUTO_Z,Z_SPEC,COADD_OBJECTS_ID
0,1,104.759602,-55.903158,0.868126,0.783947,1.0,1.000000,1.000000,21.949570,0.021828,20.347677,0.005885,19.803492,0.009052,19.489954,0.010690,-1.0,2924102426
1,1,104.804707,-55.886614,1.203799,0.032133,1.0,1.000000,0.694302,21.080441,0.009810,19.955923,0.004091,19.536043,0.007742,19.330839,0.009808,-1.0,2924104213
2,1,104.779776,-55.885828,1.086250,0.855959,1.0,1.000000,0.997875,22.052946,0.020068,20.547306,0.005846,20.020580,0.009085,19.715313,0.010825,-1.0,2924104247
3,1,104.821770,-55.883279,1.312433,0.780496,1.0,1.000000,0.047996,21.168688,0.015462,19.707525,0.004792,19.179884,0.008525,18.859488,0.009843,-1.0,2924104740
4,1,104.811481,-55.872320,1.334219,0.622183,1.0,1.000000,0.017850,21.123981,0.021484,19.915541,0.008251,19.505432,0.015854,19.218428,0.018839,-1.0,2924105979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90674,9058,67.502392,-58.370693,0.328518,0.508214,1.0,0.999999,1.000000,25.143179,0.226469,23.788034,0.076743,22.617744,0.048809,22.279425,0.067014,-1.0,2945499429
90675,9058,67.512681,-58.370300,0.416753,0.050354,1.0,0.999373,1.000000,24.359793,0.133894,23.429205,0.066992,22.679384,0.062688,22.327967,0.084922,-1.0,2945499464
90676,9058,67.515461,-58.368211,0.461415,0.138046,1.0,1.000000,1.000000,24.519791,0.117520,23.343904,0.049114,22.529673,0.041396,22.140692,0.052552,-1.0,2945499645
90677,9058,67.504254,-58.360906,0.479286,0.072485,1.0,1.000000,1.000000,23.716101,0.067762,22.338913,0.022719,21.479088,0.019162,21.159414,0.025995,-1.0,2945503332


In [3]:
bounding_boxes = pd.read_pickle("/data/scratch/des/bounding_coordinates.pickle")
bounding_boxes

{'DES0053-2041': {'RA_min': 13.056454,
  'RA_max': 13.819535,
  'DEC_min': -21.043049,
  'DEC_max': -20.329176},
 'DES0330-1250': {'RA_min': 52.283473,
  'RA_max': 53.015643,
  'DEC_min': -13.190276,
  'DEC_max': -12.476396},
 'DES2225+0001': {'RA_min': 336.037505,
  'RA_max': 336.75138,
  'DEC_min': -0.340278,
  'DEC_max': 0.373607},
 'DES0102-6205': {'RA_min': 14.901489,
  'RA_max': 16.426685,
  'DEC_min': -62.448608,
  'DEC_max': -61.734731},
 'DES0409-2458': {'RA_min': 61.867561,
  'RA_max': 62.655011,
  'DEC_min': -25.326381,
  'DEC_max': -24.612533},
 'DES0113-0624': {'RA_min': 18.087182,
  'RA_max': 18.80553,
  'DEC_min': -6.765254,
  'DEC_max': -6.05141},
 'DES0535-5248': {'RA_min': 83.30189,
  'RA_max': 84.482939,
  'DEC_min': -53.168055,
  'DEC_max': -52.454173},
 'DES0547-4957': {'RA_min': 86.431459,
  'RA_max': 87.541011,
  'DEC_min': -50.312453,
  'DEC_max': -49.598615},
 'DES0216-1207': {'RA_min': 33.651472,
  'RA_max': 34.381605,
  'DEC_min': -12.476374,
  'DEC_max': -11

In [18]:
cluster_indices = pd.unique(redmapper_catalog["ID"])
cluster_idx = np.random.choice(cluster_indices)
cluster_cat = redmapper_catalog[redmapper_catalog["ID"] == cluster_idx]
cluster_cat

,ID,RA,DEC,R,P,P_FREE,THETA_I,THETA_R,MAG_AUTO_G,MAGERR_AUTO_G,MAG_AUTO_R,MAGERR_AUTO_R,MAG_AUTO_I,MAGERR_AUTO_I,MAG_AUTO_Z,MAGERR_AUTO_Z,Z_SPEC,COADD_OBJECTS_ID
62883,1562,87.000504,-59.813630,0.780497,0.155665,1.0,0.933586,0.263120,24.458364,0.180312,22.747335,0.039885,21.913887,0.039072,21.415602,0.050432,-1.0,2947226855
62884,1562,86.963276,-59.820095,0.566301,0.792222,1.0,1.000000,0.999869,23.586781,0.102304,21.893431,0.021424,21.170717,0.026107,20.808969,0.037514,-1.0,2947226323
62885,1562,86.954604,-59.819069,0.503290,0.147513,1.0,1.000000,1.000000,23.265802,0.069303,21.515163,0.014444,20.983519,0.020484,20.661087,0.028474,-1.0,2947226389
62886,1562,86.871429,-59.809778,0.335016,0.812227,1.0,1.000000,1.000000,23.673544,0.110520,21.862041,0.020135,21.240950,0.022566,20.878721,0.030998,-1.0,2947227133
62887,1562,86.959403,-59.812018,0.471323,0.835725,1.0,1.000000,1.000000,22.096832,0.043471,20.259251,0.007945,19.488220,0.008178,19.105764,0.011678,-1.0,2947227233
62888,1562,86.914043,-59.806326,0.144325,0.485072,1.0,1.000000,1.000000,23.952913,0.158954,22.318316,0.031474,21.478228,0.024693,20.984419,0.036951,-1.0,2947227512
62889,1562,86.867348,-59.805618,0.333396,0.403511,1.0,0.999790,1.000000,24.178741,0.199141,22.616564,0.042982,21.704931,0.038204,21.313742,0.050367,-1.0,2947227558
62890,1562,86.977877,-59.804134,0.570366,0.248144,1.0,1.000000,0.999821,23.124886,0.074312,21.792444,0.021439,21.217093,0.027352,20.873587,0.042575,-1.0,2947227762
62891,1562,86.925120,-59.800990,0.153288,0.851537,1.0,1.000000,1.000000,22.798323,0.060434,21.070665,0.012839,20.313129,0.011236,19.984030,0.018804,-1.0,2947228037
62892,1562,86.928170,-59.800597,0.174706,0.731983,1.0,1.000000,1.000000,23.663622,0.135216,21.624836,0.022586,20.765951,0.018225,20.387764,0.026174,-1.0,2947228083


In [19]:
for k, v in bounding_boxes.items():
    RA_max, RA_min, DEC_max, DEC_min = v["RA_max"], v["RA_min"], v["DEC_max"], v["DEC_min"]
    RA, DEC = cluster_cat.iloc[0]["RA"], cluster_cat.iloc[3]["DEC"]
    if (RA_min < RA and RA < RA_max) and (DEC_min < DEC and DEC < DEC_max):
        DES_tile = k
        break

In [20]:
DES_tile

'DES0002-5957'

In [21]:
DES_image = "/nfs/turbo/lsa-regier/scratch/gapatron/desdr-server.ncsa.illinois.edu/despublic/dr2_tiles/DES0002-5957/DES0002-5957_r4907p01_r_nobkg.fits.fz"
from astropy.wcs import WCS
hdu_list = fits.open(DES_image)
ra_max = max(cluster_cat["RA"])
dec_max = max(cluster_cat["DEC"])
ra_min = min(cluster_cat["RA"])
dec_min = min(cluster_cat["DEC"])
w = WCS(hdu_list[1].header)
x_max, y_max = w.all_world2pix(ra_max, dec_max, 1)
x_min, y_min = w.all_world2pix(ra_min, dec_min, 1)

In [22]:
y_max

array(-411835.27238801)

In [37]:
DES_SVA_tiles = pd.read_pickle("/data/scratch/des/sva_map.pickle")

In [38]:
DES_SVA_tiles

['DES2242-4457',
 'DES0437-5540',
 'DES0457-5248',
 'DES0415-5331',
 'DES2358-4414',
 'DES0223-0333',
 'DES0511-5457',
 'DES0222-0624',
 'DES0449-4748',
 'DES0452-5540',
 'DES0111-4914',
 'DES0449-4914',
 'DES0506-4831',
 'DES0414-5040',
 'DES2327-5331',
 'DES0342-2832',
 'DES0010-5331',
 'DES2356-5705',
 'DES2324-5622',
 'DES0516-5457',
 'DES0506-4706',
 'DES0442-5205',
 'DES0456-5705',
 'DES0039-4457',
 'DES0513-5748',
 'DES0233-0458',
 'DES0224-0458',
 'DES0428-5914',
 'DES0506-6039',
 'DES0026-4331',
 'DES0001-5705',
 'DES0540-5748',
 'DES2332-5414',
 'DES0420-4623',
 'DES0332-2832',
 'DES2359-0124',
 'DES0544-5957',
 'DES0419-5205',
 'DES0446-5040',
 'DES0529-5914',
 'DES0056-4831',
 'DES0034-4414',
 'DES2330-5748',
 'DES2255-4414',
 'DES0001-4914',
 'DES2331-5248',
 'DES2251-4331',
 'DES0421-4457',
 'DES0248-0207',
 'DES0030-4331',
 'DES0448-5831',
 'DES0006-5705',
 'DES0428-4540',
 'DES2336-5748',
 'DES0109-4831',
 'DES0334-2623',
 'DES0441-4457',
 'DES0004-5248',
 'DES0410-5040

In [39]:
i = 0
for tile in DES_SVA_tiles:
    if bounding_boxes[tile]["RA_max"] - bounding_boxes["RA_min"] > 300:
        i += 1
        #print(tile)
        #print(bounding_boxes[tile])
        print(f"Removing {tile} ...")
        print(bounding_boxes[tile])
        DES_SVA_tiles.remove(tile)
        

Removing DES2358-4414 ...
{'RA_min': 1e-06, 'RA_max': 359.999999, 'DEC_min': -44.601377, 'DEC_max': -43.887504}
Removing DES2356-5705 ...
{'RA_min': 358.444926, 'RA_max': 359.758959, 'DEC_min': -57.451386, 'DEC_max': -56.737505}
Removing DES0001-5705 ...
{'RA_min': 1.3e-05, 'RA_max': 359.999996, 'DEC_min': -57.451386, 'DEC_max': -56.737519}
Removing DES2359-0124 ...
{'RA_min': 1e-06, 'RA_max': 360.0, 'DEC_min': -1.768041, 'DEC_max': -1.054168}
Removing DES0001-4914 ...
{'RA_min': 1.9e-05, 'RA_max': 359.999986, 'DEC_min': -49.598609, 'DEC_max': -48.884725}
Removing DES2354-5540 ...
{'RA_min': 357.906348, 'RA_max': 359.172052, 'DEC_min': -56.023608, 'DEC_max': -55.309732}
Removing DES2359-4540 ...
{'RA_min': 2.1e-05, 'RA_max': 359.999967, 'DEC_min': -46.029159, 'DEC_max': -45.315284}
Removing DES2359-4706 ...
{'RA_min': 1.8e-05, 'RA_max': 359.999969, 'DEC_min': -47.456931, 'DEC_max': -46.743062}
Removing DES2359-5205 ...
{'RA_min': 0.0, 'RA_max': 359.999954, 'DEC_min': -52.454159, 'DEC_m

In [32]:
len(DES_SVA_tiles)

692